In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/notebook


In [2]:
# from langchain_text_splitters import NLTKTextSplitter, SpacyTextSplitter
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (BSHTMLLoader, 
                                                  DirectoryLoader, 
                                                  Docx2txtLoader, 
                                                  NewsURLLoader, 
                                                  PyPDFLoader, 
                                                  PyMuPDFLoader,
                                                  MathpixPDFLoader,
                                                  RecursiveUrlLoader, 
                                                  SeleniumURLLoader, 
                                                  TextLoader, 
                                                  UnstructuredHTMLLoader,
                                                UnstructuredImageLoader,
                                                UnstructuredPowerPointLoader, 
                                                UnstructuredURLLoader, 
                                                UnstructuredWordDocumentLoader, 
                                                YoutubeLoader)
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter, NLTKTextSplitter, SpacyTextSplitter

from bs4 import BeautifulSoup as Soup
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from unstructured.cleaners.core import clean_extra_whitespace
import os
import torch
import re
from PIL import Image

from transformers import NougatProcessor, VisionEncoderDecoderModel, pipeline
from huggingface_hub import hf_hub_download

from datasets import load_dataset

from src.utils import *

In [6]:
data = config_parser(data_config_path = 'config/data_config.yaml')
model_config = config_parser(data_config_path = 'config/model_config.yaml')

In [4]:
# embeddings=HuggingFaceEmbeddings(model_name=model_config.get('embedding_model'),
#                                         model_kwargs={'device':take_device()})

Selected device: cuda


/home/mrzaizai2k/code_Bao/LLM-with-RAG/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [42]:
# single_url_list = data.get('single_url_list')
# loader = NewsURLLoader(urls=single_url_list, 
#                     post_processors=[clean_extra_whitespace],)
# ori_text = loader.load()


In [43]:
pdf_path = "data/web_data/Chapter_3 - EERD.pdf"
loader = PyMuPDFLoader(pdf_path)
ori_text = loader.load()
ori_text = combine_short_doc(ori_text, 100)

In [44]:
ori_text[10]

Document(page_content='Specialization\n\uf06eSpecialization \n\uf071Process of defining a set of subclasses of an entity \ntype, called superclass\n\uf071Defined on the basis of some distinguishing \ncharacteristic of the entities in the superclass\n\uf071May have several specializations of the same entity \ntype based on different distinguishing characteristics \n\uf06eSubclass can have its own:\n\uf071Specific attributes (local attributes)\n\uf071Specific relationship types\n13\nJan - 2015\n', metadata={'source': 'data/web_data/Chapter_3 - EERD.pdf', 'file_path': 'data/web_data/Chapter_3 - EERD.pdf', 'page': 12, 'total_pages': 67, 'format': 'PDF 1.7', 'title': 'Course:        Database Management Systems Credits: 3', 'author': 'PHONG VU COMPUTER', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® for Office 365', 'producer': 'Microsoft® PowerPoint® for Office 365', 'creationDate': "D:20190620145018+07'00'", 'modDate': "D:20190620145018+07'00'", 'trapped': ''})

In [45]:
# processor = NougatProcessor.from_pretrained("facebook/nougat-base")
# model = VisionEncoderDecoderModel.from_pretrained("facebook/nougat-base")

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)
# # prepare PDF image for the model
# filepath = "data/web_data/image/slide1.jpg"
# image = Image.open(filepath).convert("RGB")

In [46]:
# pixel_values = processor(image, return_tensors="pt").pixel_values

# # generate transcription (here we only generate 30 tokens)
# outputs = model.generate(
#     pixel_values.to(device),
#     min_length=1,
#     max_new_tokens=1024,
#     bad_words_ids=[[processor.tokenizer.unk_token_id]],
# )

# sequence = processor.batch_decode(outputs, skip_special_tokens=True)
# print(repr(sequence))


In [47]:
# text = (repr(sequence)[1:-1])

In [48]:

# recursive_url_list = data.get('recursive_url_list')

# loader = RecursiveUrlLoader(
#     url=recursive_url_list[0], max_depth=2,
#       extractor=lambda x: Soup(x, "html.parser").text,
# )
# docs = loader.load()

In [49]:
# loader = Docx2txtLoader("web_data/sample4.docx")
# data = loader.load()
# print(data)

In [50]:
# urls = [
# "https://e.vnexpress.net/news/travel-guide/binh-phuoc-boasts-jungle-treks-charming-feasts-4673392.html",
# 'https://e.vnexpress.net/news/culture/vietnamese-comic-artist-wins-silver-at-int-l-manga-awards-4695364.html',
# ]
# loader = UnstructuredURLLoader(urls=urls, 
#                     post_processors=[clean_extra_whitespace],)
# data = loader.load()
# print(data)
#

In [51]:
# youtube_url_list = data.get('youtube_url_list')

# loader = YoutubeLoader.from_youtube_url(
#     youtube_url_list[0], add_video_info=False
# )
# data = loader.load()
# print(data)

In [52]:
   
class NewsSummarizer:
    def __init__(self, summarizer = pipeline("summarization", 
                                             model="Falconsai/text_summarization", 
                                             torch_dtype=torch.float16,
                                             device = take_device()),
                #  translator = GoogleTranslator(),
                 chunk_overlap:str = 10,
                 max_length:int=200, 
                 min_length:int=30,
                 ):
        self.summarizer = summarizer
        # self.translator = translator
        self.max_length = max_length
        self.min_length = min_length
        self.chunk_overlap = chunk_overlap
        self.text_splitter = self.load_text_splitter()
    
    def load_text_splitter(self):
        text_splitter =  NLTKTextSplitter(chunk_size=1000)
        return text_splitter

    def summary_text(self,text_chunks:list):
        '''Summary short text'''
        sum_text= f''
        for model_output in self.summarizer(text_chunks, batch_size=8, 
                                            truncation="only_first",):
            text = model_output['summary_text']
            sum_text += f'\n{text}'
        return sum_text
    
    def summary_news(self, news:str)->str:
        # trans_news = self.translator.translate(text=news, to_lang='en')
        text_chunks = self.text_splitter.split_text(news)
        summary_text = self.summary_text(text_chunks)
        # summary_text = self.translator.translate(text=summary_text, to_lang='vi')

        return summary_text


Selected device: cuda


In [53]:
text = ori_text[0].page_content

In [54]:
text

'Chapter 3:\nEnhanced Entity-\nRelationship (EER) Model\nJan - 2015\n Contents\n2\n1\nIntroduction to Enhanced-ER Model\n2\nSubclasses, Superclasses, and Inheritance\n3\nSpecialization and Generalization\n4\nConstraints and Characteristics of Specialization \nand Generalization Hierarchies\n5\nCategories\n6\nDesign Choices, and Formal Definitions\nJan - 2015\n'

In [55]:
len(ori_text)

53

In [67]:
for i, txt in enumerate(ori_text[30:35]):
    print(f"{txt.page_content}\n-------")

Categories (1)
Category or Union type
Represents a single superclass/subclass 
relationship with more than one superclass
Subclass represents a collection of objects that is 
a subset of the UNION of distinct entity types
Attribute inheritance works more selectively
Category can be total or partial
Some modeling methodologies do not have 
union types
39
Jan - 2015

-------
Categories (2)
40
Example: Database for vehicle registration, vehicle 
owner can be a person, a bank (holding a lien on a 
vehicle) or a company.
Category (subclass) OWNER is a subset of the union 
of the three superclasses COMPANY, BANK, and 
PERSON 
A category member must exist in only one of its 
superclasses
Note: The difference from shared subclass, which is 
a subset of the intersection of its superclasses
(shared subclass member must exist in all of its 
superclasses)
Jan - 2015

-------
Two categories (union types): 
OWNER and 
REGISTERED_VEHICLE
41
Jan - 2015
 Categories (3)
42
A category can be t

In [57]:
def check(texts):
    for i, t in enumerate(texts):
        print(f"{texts[i]}\n-----------")

In [58]:
ori_text[12]

Document(page_content='Example of a Generalization\n17\nJan - 2015\n Specialization and Generalization (1)\n18\n\uf06eDiagrammatic notation sometimes used to \ndistinguish between generalization and \nspecialization\n\uf071Arrow pointing to the generalized superclass \nrepresents a generalization \n\uf071Arrows pointing to the specialized subclasses \nrepresent a specialization \n\uf071We do not use this notation because it is often \nsubjective as to which process is more appropriate for \na particular situation \n\uf071We advocate not drawing any arrows in these \nsituations \nJan - 2015\n', metadata={'source': 'data/web_data/Chapter_3 - EERD.pdf', 'file_path': 'data/web_data/Chapter_3 - EERD.pdf', 'page': 17, 'total_pages': 67, 'format': 'PDF 1.7', 'title': 'Course:        Database Management Systems Credits: 3', 'author': 'PHONG VU COMPUTER', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® for Office 365', 'producer': 'Microsoft® PowerPoint® for Office 365', 'crea

In [59]:
text = ori_text[12].page_content
print(text)

Example of a Generalization
17
Jan - 2015
 Specialization and Generalization (1)
18
Diagrammatic notation sometimes used to 
distinguish between generalization and 
specialization
Arrow pointing to the generalized superclass 
represents a generalization 
Arrows pointing to the specialized subclasses 
represent a specialization 
We do not use this notation because it is often 
subjective as to which process is more appropriate for 
a particular situation 
We advocate not drawing any arrows in these 
situations 
Jan - 2015



In [60]:
token_text_splitter = TokenTextSplitter(chunk_size=150, chunk_overlap=10)
token_texts = token_text_splitter.split_text(text)
check(token_texts)

Example of a Generalization
17
Jan - 2015
 Specialization and Generalization (1)
18
Diagrammatic notation sometimes used to 
distinguish between generalization and 
specialization
Arrow pointing to the generalized superclass 
represents a generalization 
Arrows pointing to the specialized subclasses 
represent a specialization 
We do not use this notation because it is often 
subjective as to which process is more appropriate for 
a particular situation 
We advocate not drawing any arrows in these 
situations 
Jan - 2015

-----------


In [61]:
nltk_text_splitter = NLTKTextSplitter(chunk_size=1000, chunk_overlap=100)
nltk_texts = nltk_text_splitter.split_text(text)
check(nltk_texts)

Example of a Generalization
17
Jan - 2015
 Specialization and Generalization (1)
18
Diagrammatic notation sometimes used to 
distinguish between generalization and 
specialization
Arrow pointing to the generalized superclass 
represents a generalization 
Arrows pointing to the specialized subclasses 
represent a specialization 
We do not use this notation because it is often 
subjective as to which process is more appropriate for 
a particular situation 
We advocate not drawing any arrows in these 
situations 
Jan - 2015
-----------


In [62]:
def sum_check(text_chunks):
    sum_text = NewsSummarizer().summary_text(text_chunks)
    print(sum_text)


In [63]:
sum_check(token_texts)

Your max_length is set to 200, but your input_length is only 118. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)



Generalization and Generalization (1) 18 Diagrammatic notation sometimes used to distinguish between generalization and specialization Arrow pointing to the generalized superclass represents a generalization . We do not use this notation because it is often subjective as to which process is more appropriate for a particular situation .


In [64]:
sum_check(nltk_texts)

Your max_length is set to 200, but your input_length is only 118. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)



Generalization and Generalization (1) 18 Diagrammatic notation sometimes used to distinguish between generalization and specialization Arrow pointing to the generalized superclass represents a generalization . We do not use this notation because it is often subjective as to which process is more appropriate for a particular situation .
